# Морфология
В этом ноутбуке описана подготовка данных для задачи POS-tagging. А также пара простых моделей на keras, решающих данную задачу. Оригинальная задача и ноутбук есть в контесте: https://www.kaggle.com/c/rupos2018/overview

## Часть 1. Загрузка корпуса
Здесь мы прочитаем корпуса из csv и разложим их по спискам.

In [1]:
# для совместимости со вторым питоном
from __future__ import print_function
import io

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pwd

'/content'

In [4]:
import os

# cd to root of repository if needed
if os.getcwd().endswith('/content'):
    os.chdir('/content/drive/My Drive/Colab Notebooks/NLP/2 Lecture/seminar')

In [5]:
# Имена файлов с данными.
TRAIN_FILENAME = "data/train.csv"
TEST_FILENAME = "data/test.csv"

In [6]:
# Считывание файлов.
from collections import namedtuple
WordForm = namedtuple("WordForm", "word pos gram")

def get_sentences(filename, is_train):
    sentences = []
    with io.open(filename, "r", encoding='utf-8') as r:
        # Пропускаем заголовок
        next(r)
        sentence = [] # будем заполнять список предложений
        for line in r:
            # предложения отделены по '\n'
            if len(line.strip()) == 0:
                if len(sentence) == 0:
                    continue
                sentences.append(sentence)
                sentence = []
                continue
            if is_train:
                # Формат: индекс\tномер_в_предложении\tсловоформа\tPOS#Грамемы
                word = line.strip().split("\t")[2]
                pos = line.strip().split("\t")[3].split("#")[0]
                gram = line.strip().split("\t")[3].split("#")[1]
                sentence.append(WordForm(word, pos, gram))
            else:
                word = line.strip().split("\t")[2]
                sentence.append(WordForm(word, '', ''))
        if len(sentence) != 0:
            sentences.append(sentence)
    return sentences

In [7]:
train = get_sentences(TRAIN_FILENAME, True)
test = get_sentences(TEST_FILENAME, False)

In [8]:
# Выыедем, что получилось
for wordform in train[0][:100]:
    print(wordform.word, '\t', wordform.pos, '\t', wordform.gram)

А 	 CONJ 	 _
ведь 	 PART 	 _
для 	 ADP 	 _
конкретных 	 ADJ 	 Case=Gen|Degree=Pos|Number=Plur
изделий 	 NOUN 	 Animacy=Inan|Case=Gen|Gender=Neut|Number=Plur
зачастую 	 ADV 	 Degree=Pos
нужен 	 ADJ 	 Degree=Pos|Gender=Masc|Number=Sing|Variant=Brev
монокристалл 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
не 	 PART 	 _
только 	 PART 	 _
крупный 	 ADJ 	 Case=Nom|Degree=Pos|Gender=Masc|Number=Sing
, 	 PUNCT 	 _
но 	 CONJ 	 _
и 	 PART 	 _
заданной 	 VERB 	 Aspect=Perf|Case=Gen|Gender=Fem|Number=Sing|Tense=Past|VerbForm=Part|Voice=Pass
формы 	 NOUN 	 Animacy=Inan|Case=Gen|Gender=Fem|Number=Sing
, 	 PUNCT 	 _
например 	 ADV 	 Degree=Pos
" 	 PUNCT 	 _
стакан 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
" 	 PUNCT 	 _
, 	 PUNCT 	 _
" 	 PUNCT 	 _
тройник 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
" 	 PUNCT 	 _
( 	 PUNCT 	 _
элемент 	 NOUN 	 Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing
трубопровода 	 NOUN 	 Animacy=Inan|Case=Gen|Gender=Masc|Number=Sing
) 	 PUNCT 	 

In [9]:
#запомним все уникальные слова и POS-теги в корпусе
word_set = set()
pos_set = set()
for sent in train:
    for wordform in sent:
        word_set.add(wordform.word.lower())
        pos_set.add(wordform.pos)

In [10]:
pos_to_index = {}
index_to_pos = {}
for pos in pos_set:
    pos_to_index[pos] = len(pos_to_index)
    index_to_pos[len(index_to_pos)] = pos

In [11]:
for word in list(word_set)[:10]: 
    print(word, end=', ')
print(pos_set)

аварцу, жарило, переговорами, боб, окраска, установка, смеха, стратификации, объединяет, жажда, {'AUX', 'VERB', 'X', 'SYM', 'CONJ', 'ADP', 'PART', 'NUM', 'DET', 'PROPN', 'INTJ', 'ADV', 'ADJ', 'SCONJ', 'PRON', 'NOUN', 'PUNCT'}


Для простоты далее будем использовать токены слов и POS-теги. Но чтобы определять грамматические значения нужно еще провести некоторые манипуляции с данными, описанные в оригинальном ноутубке. Мы же ограничимся только определением частей речи

## Часть 2. Подготовка эмбеддингов

Обычно в качестве признаков для обучения сеток используются словные эмбеддинги. Для этого можно скачать предобученные и сохранить их в матрицу, где в расположатся векторы эмбеддингах по индексам, соответсвующих слов

In [ ]:
#Загрузите эмбеддинги c https://nlp.stanford.edu/projects/glove/ или другие, которые вам нравятся и пропишите путь к ним
import numpy as np

word_embeddings_path = 'data/glove.6B.50d.txt'
word2idx = {}
word_embeddings = []
embedding_size = None
#Загружаем эмбеддинги
with io.open(word_embeddings_path, 'r', encoding="utf-8") as f_em:
    for line in f_em:
        split = line.strip().split(" ")
        # Совсем короткие строки пропускаем
        if len(split) <= 2:
            continue
        # Встретив первую подходящую строку, фиксируем размер эмбеддингов
        if embedding_size is None:
            embedding_size = len(split) - 1
            # Также нициализируем эмбеддинги для паддингов и неизвестных слов
            word2idx["PAD"] = len(word2idx)
            print(word2idx["PAD"])
            word_embeddings.append(np.zeros(embedding_size))

            word2idx["UNK"] = len(word2idx)
            print(word2idx["UNK"])
            word_embeddings.append(np.random.uniform(-0.25, 0.25, embedding_size))
        # После этого все эмбеддинги должны быть одинаковой длины
        if len(split) - 1 != embedding_size:
            continue
            
        #Если слова нет в корпусе, то не будем для него запоминать эмбеддинг        
        if (split[0] not in word_set):
            continue
        
        word_embeddings.append(np.asarray(split[1:], dtype='float32'))
        word2idx[split[0]] = len(word2idx)

word_embeddings = np.array(word_embeddings, dtype='float32')

0
1


In [ ]:
len(word_set & set(word2idx.keys()))

1948

In [ ]:
len(word_set)

98880

Как-то эмбеддинги не сильно подходят для данного корпуса поэтому, просто инициализируем рандмно матрицу эмбеддингов при определении сетки. Вам же предлагается все-таки поискать подходящие эмбеддинги и использовать их при обучении.

## Часть 3. Подготовка данных
Теперь нам остается только пронумеровать все слова и POS-теги и можно переходить к обучению сеток.

In [ ]:
word_to_index = {'PAD' : 0, 'UNK' : 1}
for word in word_set:
    word_to_index[word] = len(word_to_index)

In [ ]:
# для полносвязной сетки просто захреначим все в один список
data_X = []
data_Y = []
for sent in train:
    for wordform in sent:
        data_X.append(word_to_index[wordform.word.lower()])
        data_Y.append(pos_to_index[wordform.pos])

In [ ]:
word_to_index[train[0][0].word.lower()]

52022

In [ ]:
print(data_X[:10])
print(data_Y[:10])

[52022, 39483, 26638, 10439, 28808, 78693, 14795, 69957, 82974, 19738]
[12, 8, 4, 2, 13, 16, 2, 13, 8, 8]


## Часть 4. Полносвязная сеть
Самой простой моделью является обычный перцептрон. На вход сетки будем подавать просто эмдеддинг каждого слова, на выходе ожидать распредедение вероятностей по тегам. В качестве фреймворка достаточно будет использовать keras и его Sequential модель (https://keras.io/models/sequential/), в которую слои добавляются последовательно, с помощью метода `add`.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Dense, Activation, Flatten

In [ ]:
model = Sequential()
# на самом деле на вход сетки будет добавляться индекс слова, а слой эмбеддинга будет возвращать для него вектор
model.add(
    Embedding(
        input_length=1, 
        input_dim=len(word_to_index), 
        output_dim=50, 
        embeddings_initializer='random_uniform',
        trainable=False,
      )
) # матрицу эмбеддингов просто инициализируем нормальным распределением и отключим обучение
# далее нам нужно схлопнуть трехмерный тензор с одной фиктивной размерностью в двумерный
model.add(Flatten())
model.add(Dense(100)) # основной полносвязный слой
model.add(Activation('relu')) # для приличия добавим функцию активации
model.add(Dense(len(pos_to_index))) # выходной слой тоже полносвязный размерности по кол-ву тегов
model.add(Activation('softmax')) # ну и в конце делаем softmax, чтобы получить распределение
model.summary() # вывод получившейся модели

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1, 50)             4944100   
_________________________________________________________________
flatten_1 (Flatten)          (None, 50)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
activation_2 (Activation)    (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 17)                1717      
_________________________________________________________________
activation_3 (Activation)    (None, 17)                0         
Total params: 4,950,917
Trainable params: 6,817
Non-trainable params: 4,944,100
________________________________________

In [ ]:
# компилируем модель
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'],
    )

In [ ]:
# и обучаем
model.fit(np.array(data_X), np.array(data_Y), epochs=1, batch_size=256)

3324/3324 [==============================] - 6s 2ms/step - loss: 1.4159 - accuracy: 0.5468


Проверка обученности модели остается за вами. Этот пример лишь для того, чтобы показать как собрать сетку и скормить ей данные.

## Часть 5. Рекуррентая сеть.

Далее рассмотрим более приближенную к SOTA модель. Ей является рекуррентая сеть, которая принимает эмбеддинги слов в предложении и генерирует для них распределение вероятностей. Основным отличием от прошлой в том, что теперь мы будем использовать соседние слова как раз за счет рекуррентого слоя. Для этой модели мы уже будем использовать функциональный способ задания модели все того же кераса (https://keras.io/models/model/).

In [ ]:
from keras.layers import LSTM, TimeDistributed,Bidirectional, Input
from keras.models import Model

In [ ]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(None,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(
    input_dim=len(word_to_index),
    output_dim=50, 
    trainable=False, 
    embeddings_initializer='random_uniform',
    name='embedding',
)(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
# return_sequences в каждоый момент времени, иначе смотрим на последний state
blstm_layer = Bidirectional(LSTM(100, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
# Выдает ответ в каждый момент времени
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(blstm_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# выводим архитектуру
model.summary()
# на вход батча на размер предложения
# на вход батча на размер предложения и размерности слова
# на вход батча на размер предложения и 100 LSTM
# Все предлжения до одинакого размера

Model: "functional_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 50)          4944100   
_________________________________________________________________
blstm (Bidirectional)        (None, None, 200)         120800    
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 17)          3417      
Total params: 5,068,317
Trainable params: 124,217
Non-trainable params: 4,944,100
_________________________________________________________________


Далее нам нужно было бы распределить слова по предложениям, распределить по группам по длине, выравнить предложения по длине в одной групе, заполнив недостающие слова паддингами. Но это довольно неприятный процесс, а мне просто хочется запустить сетку и проверить, что она вообще работает, что сошлись все разверности. Поэтому просто раскидаем по 10 слов с помощью `numpy.reshape`

In [ ]:
rnnX = np.reshape(data_X[:850000], (-1,10))
rnnY = np.reshape(data_Y[:850000], (-1,10,1))

In [ ]:
rnnY[0].shape

(10, 1)

In [ ]:
rnnX.shape, rnnY.shape

((85000, 10), (85000, 10, 1))

In [ ]:
np.shape(rnnX)

(85000, 10)

Ну и проверим, что оно вообще работает.

In [ ]:
model.fit(rnnX, rnnY, epochs=1, batch_size=256)

333/333 [==============================] - 24s 73ms/step - loss: 2.0793 - accuracy: 0.3594


## Часть 6. Задание
В качестве упражения предлагается довести до ума обучения второй модели: распределить слова по предложениям, написать тестирование модели и собственно посмотреть как оно обучилось. Тестировать предлагаю на последней 1000 предложений, обучать - на остальном. Кто уверен в своих желаниях, то может решить оригинальную задачу: предсказывать также грамматические категории. 

In [12]:
from keras.layers import LSTM, TimeDistributed,Bidirectional, Input, BatchNormalization, Embedding, Dense, Activation
from keras.models import Model

In [13]:
%%time
import numpy as np

word_embeddings_path = 'data/cc.ru.300.vec'
word2idx = {}
word_embeddings = []
embedding_size = None
#Загружаем эмбеддинги
with io.open(word_embeddings_path, 'r', encoding="utf-8") as f_em:
    for line in f_em:
        split = line.strip().split(" ")
        # Совсем короткие строки пропускаем
        if len(split) <= 2:
            continue
        # Встретив первую подходящую строку, фиксируем размер эмбеддингов
        if embedding_size is None:
            embedding_size = len(split) - 1
            # Также нициализируем эмбеддинги для паддингов и неизвестных слов
            word2idx["PAD"] = len(word2idx)
            print(word2idx["PAD"])
            word_embeddings.append(np.zeros(embedding_size))

            word2idx["UNK"] = len(word2idx)
            print(word2idx["UNK"])
            word_embeddings.append(np.random.uniform(-0.25, 0.25, embedding_size))
        # После этого все эмбеддинги должны быть одинаковой длины
        if len(split) - 1 != embedding_size:
            continue
            
        #Если слова нет в корпусе, то не будем для него запоминать эмбеддинг        
        if (split[0] not in word_set):
            continue
        
        word_embeddings.append(np.asarray(split[1:], dtype='float32'))
        word2idx[split[0]] = len(word2idx)

word_embeddings = np.array(word_embeddings, dtype='float32')

0
1
CPU times: user 56 s, sys: 2.7 s, total: 58.7 s
Wall time: 1min 22s


In [14]:
len(word_set & set(word2idx.keys()))

89408

In [15]:
def sentences_to_indices(data, word_to_index, max_len):
    """
    Params:
        data
        word_to_index
        pos_to_index
        max_len 
    """

    m = len(data)
    X_indices = np.zeros((m, max_len), dtype=int)
    
    for i, sentence in enumerate(data):
      for j, wordform in enumerate(sentence[:max_len]):
          X_indices[i, j] = word_to_index.get(wordform.word.lower(), word_to_index['UNK'])

    return X_indices

def pos_to_indices(data, pos_to_index, max_len):
    """
    Params:
        data
        pos_to_index
    """
    
    m = len(data)
    y_indices = np.zeros((m, max_len, 1), dtype=int)
    for i, sentence in enumerate(data):
      for j, wordform in enumerate(sentence[:max_len]):
          y_indices[i, j, 0] = pos_to_index[wordform.pos]

    return y_indices

In [16]:
max_len = 20

X_train = sentences_to_indices(train[:-1000], word2idx, max_len)
X_valid = sentences_to_indices(train[-1000:], word2idx, max_len)

y_train = pos_to_indices(train[:-1000], pos_to_index, max_len)
y_valid = pos_to_indices(train[-1000:], pos_to_index, max_len)

In [17]:
X_train.shape, y_train.shape

((47171, 20), (47171, 20, 1))

In [18]:
X_train.max()

89409

In [19]:
# В начале задается входной слой, в котором мы укажем входную размерность. 
# Это будет None, т.к. мы заранее не знаем, какой будет длина каждого предложения 
input_layer = Input(shape=(max_len,), name='input')
# Далее идет все тот же слой эмеддинга, которому мы на вход подаем предыдущий слой (в этом и суть functional APO)
embeddings_layer = Embedding(
    input_dim=word_embeddings.shape[0],
    output_dim=word_embeddings.shape[1], 
    trainable=True, 
    weights=[word_embeddings],
    # embeddings_initializer='random_uniform',
    name='embedding',
)(input_layer)
# Итак, основным слоем здесь будет двусторонний LSTM, который будет возвращать вектор для каждого слова (return_sequences=True) 
# return_sequences в каждоый момент времени, иначе смотрим на последний state
blstm_layer = Bidirectional(LSTM(128, return_sequences=True), name='blstm')(embeddings_layer)
# Аналогично т.к. у нас здесь вектора для каждого слоя, то и полносвязный слой должен применяться для каждого слоя 
# по-отдельности. Поэтому полносвязный слой оборачивается в  TimeDistributed
# Выдает ответ в каждый момент времени
dense_layer = TimeDistributed(Dense(128, name='dense'))(blstm_layer)
bacth_norm = TimeDistributed(BatchNormalization(axis=-1))(dense_layer)
relu_layer = TimeDistributed(Activation('relu', name='relu'))(bacth_norm)
# Выдает ответ в каждый момент времени
result_layer = TimeDistributed(Dense(len(pos_to_index), activation='softmax', name='result'))(relu_layer)
# собственно определяем модель входными и выходными слоями
model = Model(inputs=[input_layer], outputs=result_layer)
# компилируем
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'],
)
# выводим архитектуру
model.summary()
# на вход батча на размер предложения
# на вход батча на размер предложения и размерности слова
# на вход батча на размер предложения и 100 LSTM
# Все предлжения до одинакого размера

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 20)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 20, 300)           26823000  
_________________________________________________________________
blstm (Bidirectional)        (None, 20, 256)           439296    
_________________________________________________________________
time_distributed (TimeDistri (None, 20, 128)           32896     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 20, 128)           512       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 20, 128)           0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 20, 17)           

In [20]:
model.fit(X_train, y_train, epochs=5, batch_size=256)

Epoch 1/5
185/185 [==============================] - 115s 619ms/step - loss: 0.3063 - accuracy: 0.9122
Epoch 2/5
185/185 [==============================] - 114s 618ms/step - loss: 0.0577 - accuracy: 0.9811
Epoch 3/5
185/185 [==============================] - 115s 619ms/step - loss: 0.0344 - accuracy: 0.9890
Epoch 4/5
185/185 [==============================] - 114s 614ms/step - loss: 0.0183 - accuracy: 0.9946
Epoch 5/5
185/185 [==============================] - 113s 613ms/step - loss: 0.0098 - accuracy: 0.9974


In [21]:
# Score the model
y_pred = model.predict(X_valid)

In [22]:
np.mean(np.argmax(y_pred, axis=-1).reshape(-1) == y_valid.reshape(-1))

0.9732